In [20]:
from pyecharts import options as opts
from pyecharts.charts import Line, Timeline, Bar, Scatter, WordCloud
from pyecharts.commons.utils import JsCode
from pyecharts.globals import SymbolType
import pandas as pd
import numpy as np
import jieba
import re

In [ ]:
# 清洗后的数据所在路径
cleaned_data_path = r'C:\Users\12517\PycharmProjects\NLP\projects\taptap评论情感分析\data\tap_reviews cleaned.csv'
# 停用词txt路径（后面会用到）
stop_word_path = r'C:\Users\12517\PycharmProjects\NLP\projects\taptap评论情感分析\data\stop_words.txt'

In [21]:
# 加载数据
data = pd.read_csv(cleaned_data_path, encoding='utf_8_sig')

In [22]:
"""
玩家评分变化趋势——一定程度上反映玩家活跃度变化和对版本的满意度
"""
# 数据透视表，对日期聚合，计算每日评分的均值、频次
table = pd.pivot_table(data, index=['updated_time'], values=['score'], aggfunc=['mean', 'count'])
table.columns = ['score_mean', 'score_count']
table['score_mean'] = np.round(table['score_mean'], 2)
# 将index转换为DatetimeIndex格式，方便后面画图
table.index = pd.date_range(str(table.index[0]), str(table.index[-1]), freq='D')

In [23]:
# 评分趋势折线
data_type = {'评分均值': 'score_mean', '评分频率': 'score_count'}
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#db5856'}, {offset: 1, color: '#0b8b87'}], false)"
)
time_line = Timeline(init_opts=opts.InitOpts(theme='dark', width='400'))
time_line.add_schema(is_auto_play=True, play_interval=10000)
for key_, value_ in data_type.items():
    line = (
        Line(init_opts=opts.InitOpts(theme='dark'))
        .add_xaxis(table.index)
        .add_yaxis('玩家评分', table[value_], is_smooth=True, areastyle_opts=opts.AreaStyleOpts(
            color=JsCode(area_color_js), opacity=1), markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(
            type_='average')], linestyle_opts=opts.LineStyleOpts(color='white', type_='dashed')))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False), markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_='max', name='最高分'), opts.MarkPointItem(type_='min', name='最低分')]))
        .set_global_opts(title_opts=opts.TitleOpts(title='taptap评分趋势', subtitle='2019-09-15 22时 至 2020-03-18 13时',
            title_textstyle_opts=opts.TextStyleOpts(
            color="#fff", font_size=16)), xaxis_opts=opts.AxisOpts(type_='time'), yaxis_opts=opts.AxisOpts(
            type_='value', min_=8), tooltip_opts=opts.TooltipOpts(trigger='axis', axis_pointer_type='cross')))
    time_line.add(line, key_)
time_line.render_notebook()
# 1月30日之后，评分震荡区间上升，但仍未回到上年12月中旬前的水平
# 最低分点评论的关键词：简单，太肝，类型没意思，Bug
# 采集的数据的时间区间内，游戏分别于2019-10-17，2019-11-04，2020-01-01，2020-03-03进行版本更新，第三个版本满意度相对低，拉低了整体评价
# 自从1月9日出现评论高峰后，玩家评论的活跃度比之前提高近一倍

In [24]:
"""
点评次数前15的设备——活跃玩家使用的机型，应重点关注
"""
# 标记负向评价，这里设定score<8为负向
data['negative'] = data['score'].apply(lambda x: 1 if x < 8 else 0)
# 数据透视表，对设备聚合，计算各设备评价总数、负向评价数、游玩时间
table = pd.pivot_table(data, index=['device'], values=['score', 'negative', 'spent'], aggfunc={
    'score': 'count', 'negative': np.sum, 'spent': 'mean'}, fill_value=0)
table.columns = ['negative_count', 'device_count', 'spent_mean']
sub_table1 = table.sort_values('device_count', ascending=False)[:15]
sub_table1['positive_count'] = sub_table1['device_count'] - sub_table1['negative_count']
sub_table1['spent_mean'] /= 60

In [25]:
# 评论数量堆叠柱状图&单人游玩时间折线
bar = (Bar(init_opts=opts.InitOpts(theme='dark'))
       .add_xaxis(sub_table1.index.tolist())
       .add_yaxis('正向点评', sub_table1['positive_count'].tolist(), stack='stack1', category_gap='40%', yaxis_index=1)
       .add_yaxis('负向点评', sub_table1['negative_count'].tolist(), stack='stack1', category_gap='40%', yaxis_index=1)
       .extend_axis(yaxis=opts.AxisOpts(name='点评次数', type_='value', min_=0, max_=250, position='right',
                                        axislabel_opts=opts.LabelOpts(formatter='{value} 次')))
       .extend_axis(yaxis=opts.AxisOpts(type_='value', position='left',
                                        axislabel_opts=opts.LabelOpts(formatter='{value} 小时'),
                                        splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.
                                                                          LineStyleOpts(opacity=1))))
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
       .set_global_opts(title_opts=opts.TitleOpts(title='taptap评论最活跃的设备TOP15', subtitle='2019-09-15 22时 至 2020-03-18 13时'),
                        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=25, font_size=11)),
                        datazoom_opts=opts.DataZoomOpts(), tooltip_opts=opts.TooltipOpts(trigger='axis',
                        axis_pointer_type='cross'))
       )
line2 = (Line(init_opts=opts.InitOpts(theme='dark'))
         .add_xaxis(sub_table1.index.tolist())
         .add_yaxis('单人游玩时长', np.round(sub_table1['spent_mean'].tolist(), 2), yaxis_index=2)
         )
bar.overlap(line2).render_notebook()
# 小米和华为瓜分前15，可考虑重点关注这些机型的优化，或者和产商联合策划活动

In [26]:
"""
评分较低且评分数量大于20的设备TOP10——满意度较低的玩家，提高这些玩家的评价会很有帮助
* 可以进一步考虑评分和设备的相关性，如相关，可以考虑低评价机型的共性进行优化
"""
# 数据透视表，对设备聚合，计算评分均值、设备数量
table = pd.pivot_table(data, index=['device'], values=['score'], aggfunc={'score': ['mean', 'count']})
table.columns = ['device_count', 'score_mean']
sub_table2 = table.sort_values('score_mean')
sub_table2 = sub_table2[sub_table2['device_count'] >= 20][:10].sort_values('device_count')

In [27]:
# 散点图
scatter = (
    Scatter(init_opts=opts.InitOpts(theme='dark'))
    .add_xaxis(sub_table2.index.tolist())
    .add_yaxis('玩家评分',
               [list(z) for z in zip(np.round(sub_table2['score_mean'], 2), sub_table2.index, sub_table2['device_count'])],
               label_opts=opts.LabelOpts(formatter=JsCode("function(params){return params.value[3]+'人：'+params.value[1];}")),
               symbol_size=20)
    .set_global_opts(title_opts=opts.TitleOpts(
        title='taptap评分较低的设备TOP10', subtitle='2019-09-15 22时 至 2020-03-18 13时，只统计评论数量>=20的设备'),
        yaxis_opts=opts.AxisOpts(type_='value', min_=6.5), visualmap_opts=opts.VisualMapOpts(
        type_='color', max_=sub_table2['device_count'].max(), min_=sub_table2['device_count'].min()),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=20, font_size=11)), datazoom_opts=opts.DataZoomOpts())
)
scatter.render_notebook()
# 评论数>=20且打低分的机型较有代表性，可考虑共性并进行优化

In [28]:
"""
词云图（TOP500支持度和热度达到0.5的评论）——玩家最关注点和对游戏的直观印象

需要做以下几步（含一些文本挖掘的预处理程序）：
1.将文本合并成一个字符串 2.去除非中文字符 3.进行中文分词  4.去除中文停用词  5.统计词频  6.用pyechart绘图
"""
# 去除非中文字符
def drop_non_chinese(text):
    """
    :param text: str,含非中文的字符串
    :return: str,不含非中文的字符串
    """
    # \u4e00-\u9fa5是中文字符的范围，中括号内^表示取反
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese_text = re.sub(pattern, '', text)
    return chinese_text

# 中文分词
def split_text(chinese_text, cut_all=False):
    """
    :param chinese_text: str,中文文本
    :param cut_all: 是否采用全模式分词
    :return: str，用空格分隔的分词文本
    """
    text_generater = jieba.cut(chinese_text, cut_all=cut_all)
    result = ' '.join(text_generater)
    return result

# 去除中文停用词
def drop_stop_words(st_path, chinese_text):
    """
    :param st_path: str，停用词文件路径
    :param chinese_text: str，中文分词文本
    :return: list，去除停用词的中文分词列表
    """
    # 载入停用词列表
    with open(st_path, 'r') as f:
        st = f.read()
        st_list = st.splitlines()
    word_list = chinese_text.split()

    # 删除停用词
    for stop_word in st_list:
        word_list = [word for word in word_list if word != stop_word]

    return word_list

# 统计词频
def caculate_words(word_list, threshold=50):
    """
    :param word_list: list，去除停用词的中文分词列表
    :param threshold: int，阈值，满足词频>=阈值则保留分词
    :return: list, 每个元素为tuple,包含词语和词频，形如('现在', 50)
    """
    # 利用字典的特性计算词频
    count_dict = {}
    for word in word_list:
        count_dict[word] = count_dict[word] + 1 if word in count_dict else 1

    count_dict = {k: v for k, v in count_dict.items() if v >= threshold}
    keyword_list = list(count_dict.items())

    return keyword_list

In [29]:
# 截取并合并评论
data = data.sort_values('net_support', ascending=False)
data.reset_index(inplace=True)
sub_contents = data['contents'][(data.index<500)|(data['heat']>=0.5)]
text = str()
for content in sub_contents:
    text += content
print('文本长度', len(text))

文本长度 133202


In [18]:
# 去除非中文字符
text = drop_non_chinese(text)
# 中文分词
text = split_text(text, cut_all=False)
# 去除中文停用词
word_list = drop_stop_words(stop_word_path, text)
# 统计词频
keyword_list = caculate_words(word_list, threshold=50)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\12517\AppData\Local\Temp\jieba.cache
Loading model cost 0.719 seconds.
Prefix dict has been built successfully.


In [32]:
# 画词云图
cloud = (WordCloud()
         .add('', keyword_list, word_size_range=[20, 100])
         )
cloud.render_notebook()
# 从词云图中，可以看到玩家对游戏的关键评价和直观印象